In [ ]:
import typing
import csv
import json
import requests
import time
from email_validator import validate_email, EmailNotValidError

In [ ]:
CSV_FILENAME = 'some.csv'

FROM_EMAIL = 'someemail'
FROM_NAME = 'somename'
EMAIL_SUBJECT = 'somesubject'

In [ ]:
# Сюда пихаем аттачменты, кодированные в base64
ATT_1 = 'somebase64attachment'

In [ ]:
'''
Класс, описывающий вхождение в таблицу
'''
class Entry():
    name = str()
    email = str()
    login = str()
    password = str()
    is_valid = bool()
    
    def __init__(self, name, email, login, password):
        try:
            v = validate_email(email)
        except EmailNotValidError as e:
            is_valid = False
            self.email = email
        else:
            self.is_valid = True
            self.email = email
        finally:
            self.name = name
            self.password = password
            if login != None:
                self.login = login
            else:
                self.is_valid = False
    
    def __str__(self):
        return f'Entry for {self.email} is {"valid" if self.is_valid else "not valid"}'

In [ ]:
'''
Функция для загрузки получателей из csv
Возвращает массив объектов класса Entry
'''
def load_recipients(filename):
    recipients = []

    fieldnames = ['name','email', 'login', 'password']
    with open(filename, newline='') as csvfile:
        reader = csv.DictReader(csvfile, delimiter=';', fieldnames=fieldnames)
        for row in reader:
            recipient = Entry(
                name=row['name'],
                email=row['email'],
                login=row['login'],
                password=row['password']
            )
            recipients.append(recipient)
    return recipients

In [ ]:
'''
Класс, описывающий сообщение
'''
class Message():
    from_email = FROM_EMAIL
    from_name = FROM_NAME
    subject = EMAIL_SUBJECT
    
    recipient = None
    is_sent = bool()
    
    def __init__(self, entry):
        # Если запись помечена как валидная
        if entry.is_valid:
            self.recipient = entry
        else:
            raise ValueError()

    def generate_body(self):
        return (
            # Здесь шаблон письма, можно подставить любые переменные объекта recipient
            f'Some template here'
        )
    def to_dict(self):
        return {
            "from_email": self.from_email,
            "from_name": self.from_name,
            "subject": self.subject,
            "recipients": [{"email": self.recipient.email}],
            "body": {
                "html": self.generate_body(),
            },
            "attachments": [
                    {
                        "type": "application/pdf",
                        "name": "attachment_name.pdf",
                        "content": ATT_INSTR_V2
                    },
            ],
        }

In [ ]:
messages = []
recipients = load_recipients(CSV_FILENAME)
#recipients[0].is_valid = False
#recipients[1].is_valid = False
for i in recipients:
    try:
        message = Message(i)
    except ValueError:
        print(f'{i.email} is invalid, email will not be sent')
    else:
        messages.append(Message(i))

In [ ]:
class UnioneClient():
    api_endpoint = 'https://one.unisender.com'
    
    api_key = str()
    username = str()
    
    def __init__(self, api_key, username):
        self.api_key = api_key
        self.username = username
        
    def send(self, message: dict):
        url = f'{self.api_endpoint}/ru/transactional/api/v1/email/send.json'
        payload = {
            "api_key": self.api_key,
            "username": self.username,
            "message": message
        }
        
        req = json.dumps(payload, indent=4, ensure_ascii=False)
        
        r = requests.post(url, json=payload)

        return r

In [ ]:
client = UnioneClient('unioneapikey', 'unioneusername')

In [ ]:
responces = []
for message in messages:
    a = client.send(message.to_dict())
    if a.status_code == requests.codes.ok:
        print('Sent')
    else:
        print('Error')
    responces.append(a)

In [ ]:
responces_jsons = []
for i in responces:
    responces_jsons.append(i.text + '\n')

In [ ]:
with open('results.log', 'a') as logfile:
    for i in responces_jsons:
        logfile.write(i)